In [4]:
import cv2
import os
import numpy as np
import random

In [5]:
# Data augmentation parameters
augmentation_factor = 5  # Number of augmented samples to generate per original sample
output_folder = 'augmented_images'  # Folder to save augmented images

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# Image augmentation function
def augment_image(image):
    augmented_images = []
    
    # Invert image
    inverted_image = cv2.bitwise_not(image)
    augmented_images.append(('inverted', inverted_image))
    
    # Rotate image
    angle = random.uniform(-30, 30)
    rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    augmented_images.append(('rotated', rotated_image))
    
    # Translate image
    translation_x = random.uniform(-20, 20)
    translation_y = random.uniform(-20, 20)
    translation_matrix = np.float32([[1, 0, translation_x], [0, 1, translation_y]])
    translated_image = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]))
    augmented_images.append(('translated', translated_image))
    
    # Grayscale conversion
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    augmented_images.append(('grayscale', grayscale_image))
    
    return augmented_images

In [6]:
# Image augmentation for each folder in the dataset
data_dir = 'chest_xray/chest_xray/train'
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    
    if not os.path.isdir(folder_path):
        continue  # Skip non-directory files
    
    # Create folder for augmented images
    output_folder_path = os.path.join(output_folder, folder)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        image = cv2.imread(image_path)
        
        augmented_images = augment_image(image)
        
        # Save augmented images
        for i, (augmentation_type, augmented_image) in enumerate(augmented_images):
            output_filename = f"{filename.split('.')[0]}_{i+1}_{augmentation_type}.{filename.split('.')[1]}"
            output_path = os.path.join(output_folder_path, output_filename)
            
            cv2.imwrite(output_path, augmented_image)
